# Atténuation

Voici les résultats qu'on pourra avoir en utilisant le ratio de couverture des secteurs angulaires réels et théoriques pour l'éclairage.

A titre de comparaison, on verra l'effet "niveaux de gris" limités à COLOR_RANGE+1 couleurs différentes par rapport à l'utilisation de l'échelle complète disponible avec le RGB 8 bits.

**[R1 attention]:** il manque certains contrôles de cohérence sur les valeurs des curseurs (graphs interactifs).

**[R1 attention2]:** il y a un **bug** dans l'échantillonnage des niveaux de gris avec certaines valeurs... correction à venir.

**attention3:** Les widgets ne fonctionnent pas avec nbviewer (ou en tout cas je n'ai pas réussi à les faire fonctoinner. Si vous voulez tester ça sur votre propre serveur Jupyter, passez la valeur de **INTERACTIVE** à **True** dans la première ligne du code de la cellule suivante.

## 1. Initialisation

In [1]:
INTERACTIVE = False

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML

# NOTE: array[y, x]
print("Imports: OK...")

Imports: OK...


In [3]:
# Constants
# size of arrays
WIDTH = 10
HEIGHT = 10

# if abs(a - b) < EPSILON then a == b
EPSILON = 0.1
# For approximation
ROUNDING_POSITION = 3

# Constants
# COLOR_RANGE+1 shades of grey
COLOR_RANGE = 10
COLOR_MIN = 40
COLOR_MAX = 200
COLOR_DELTA = int((COLOR_MAX - COLOR_MIN) / COLOR_RANGE)

# x axis
vx = np.array([1., 0.])

light_angles = np.array([-180., 180.])

# Display Array, descending y axis
def dumpdata(msg, data):
    print(f"## {msg} ##\n {pd.DataFrame(data).sort_index(ascending=False, axis=0)}\n")

print("Constants and basic functions: OK...")

Constants and basic functions: OK...


In [4]:
# base angles
base_alpha = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_min = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_max = np.zeros([HEIGHT, WIDTH])*np.nan

# Compute base angles (vertical walls)
base_alpha_v = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_min_v = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_max_v = np.zeros([HEIGHT, WIDTH])*np.nan

# Compute base angles (horizontal walls)
base_alpha_h = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_min_h = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_max_h = np.zeros([HEIGHT, WIDTH])*np.nan

# real angles
real_alpha_min = np.zeros([HEIGHT, WIDTH])*np.nan
real_alpha_max = np.zeros([HEIGHT, WIDTH])*np.nan
real_alpha = np.zeros([HEIGHT, WIDTH])

# [0. ; 1.] ratio = entering light angle / base light angle
# are we receiving all the lighting that we can accept without blocking element?
real_ratio = np.zeros([HEIGHT, WIDTH])

# Blocking blocks
blocks = np.zeros([HEIGHT, WIDTH])

# vibility to color convertion
lighting_color = np.zeros([HEIGHT, WIDTH])

# Light Source L (0.5, 0.5)
L = np.array([0.5, 0.5])

print("Structures: OK")

Structures: OK


In [17]:
def resetArrays():
    real_alpha_min.fill(np.nan)
    real_alpha_max.fill(np.nan)
    real_alpha.fill(0.)
    real_ratio.fill(0.)
    real_alpha_min[0, 0] = light_angles[0]
    real_alpha_max[0, 0] = light_angles[1]
    real_alpha[0, 0] = light_angles[1] - light_angles[0]
    real_ratio[0, 0] = 1.
    blocks.fill(0.)

# Get the intersection of 2 angles
def intersect(in_min, in_max, th_min, th_max):
    if (in_max < th_min or in_min > th_max):
        # No overlap
        return (0., 0.)
    r_max = min(th_max, in_max)
    r_min = max(th_min, in_min)
    if (r_max - r_min < EPSILON):
        r_max = r_min
    return (r_min, r_max)

# Get the union of 2 angles IF they touch each other
def union(a1_min, a1_max, a2_min, a2_max):
    # One angle is null
    if (a1_min == a1_max == 0):
        return(a2_min, a2_max)
    if (a2_min == a2_max == 0):
        return(a1_min, a1_max)
    # Not neighbors ?
    if (abs(a2_min - a1_max) > EPSILON and abs(a1_min - a2_max) > EPSILON):
        #print(a1_min, a1_max, a2_min, a2_max, abs(a2_min - a1_max))
        print(f"[union] - FATAL - ({x}, {y}) ({a1_min}, {a1_max}) not neighbor of ({a2_min}, {a2_max})")
        return (0., 0.)
    # Union
    return (min(a1_min, a2_min), max(a1_max, a2_max))

def setBaseAlpha(x, y, a, b):
    alpha = np.degrees(np.math.atan2(np.linalg.det([a,b]),np.dot(a,b)))
    beta = np.degrees(np.math.atan2(np.linalg.det([vx, a]), np.dot(vx, a)))
    base_alpha[y, x] = alpha
    base_alpha_min[y, x] = beta
    base_alpha_max[y, x] = beta + alpha

def computeBaseSector():
    # First line
    y = 0
    for x in range(1,WIDTH):
        a = np.array([x, 0]) - L
        b = np.array([x, 1]) - L
        setBaseAlpha(x, y, a, b)
    
    # First Column
    x = 0
    for y in range(1, HEIGHT):
        a = np.array([1, y]) - L
        b = np.array([0, y]) - L
        setBaseAlpha(x, y, a, b)

    # All the other cells
    for y in range(1, HEIGHT):
        for x in range(1, WIDTH):
            a = np.array([x+1, y]) - L
            b = np.array([x, y+1]) - L
            setBaseAlpha(x, y, a, b)

def getAlphaBeta(a, b):
    alpha = np.degrees(np.math.atan2(np.linalg.det([a,b]),np.dot(a,b)))
    beta = np.degrees(np.math.atan2(np.linalg.det([vx, a]), np.dot(vx, a)))
    return(alpha, beta)

def setBaseAlphaH(x, y, a, b):
    (alpha, beta) = getAlphaBeta(a, b)
    base_alpha_h[y, x] = alpha
    base_alpha_min_h[y, x] = beta
    base_alpha_max_h[y, x] = beta + alpha

def setBaseAlphaV(x, y, a, b):
    (alpha, beta) = getAlphaBeta(a, b)
    base_alpha_v[y, x] = alpha
    base_alpha_min_v[y, x] = beta
    base_alpha_max_v[y, x] = beta + alpha

def computeBaseSector_Walls():
    # First line
    y = 0
    for x in range(1,WIDTH):
        a = np.array([x, 0]) - L
        b = np.array([x, 1]) - L
        setBaseAlphaH(x, y, a, a)
        setBaseAlphaV(x, y, a, b)
    
    # First Column
    x = 0
    for y in range(1, HEIGHT):
        a = np.array([1, y]) - L
        b = np.array([0, y]) - L
        setBaseAlphaH(x, y, a, b)
        setBaseAlphaV(x, y, a, a)

    # All the other cells
    for y in range(1, HEIGHT):
        for x in range(1, WIDTH):
            a = np.array([x+1, y]) - L
            b = np.array([x, y]) - L
            setBaseAlphaH(x, y, a, b)
            a = np.array([x, y]) - L
            b = np.array([x, y+1]) - L
            setBaseAlphaV(x, y, a, b)

def setRealAlpha(x, y, a_min, a_max, blocktype=""):
    real_alpha_min[y, x] = a_min
    real_alpha_max[y, x] = a_max
    real_alpha[y, x] = a_max - a_min
    # Watch out for the 1.0 -> 0.9999... 1.0 effect
    if (blocktype=="v"):
        real_ratio[y, x] = round(real_alpha[y, x] / base_alpha_v[y, x], ROUNDING_POSITION)
    elif (blocktype=="h"):
        real_ratio[y, x] = round(real_alpha[y, x] / base_alpha_h[y, x], ROUNDING_POSITION)
    elif (blocktype=="a"):
        real_ratio[y, x] = round((real_ratio[y-1, x] + real_ratio[y, x-1])/2., ROUNDING_POSITION)
    else:
        real_ratio[y, x] = round(real_alpha[y, x] / base_alpha[y, x], ROUNDING_POSITION)
    # with the ratio we know if the blocking cell receive light (obstacle must be displayed)
    if (blocks[y, x] == 1):
        real_alpha[y, x] = 0.
        real_alpha_min[y, x] = 0.
        real_alpha_max[y, x] = 0.
    return(real_alpha_min[y, x], real_alpha_max[y, x])

def computeSector(x, y):
    # Already done
    if (not np.isnan(real_alpha_min[y, x]) and not np.isnan(real_alpha_max[y, x])):
        return (real_alpha_min[y, x], real_alpha_max[y, x])
    # Origin
    if (x == 0 and y == 0):
        return (real_alpha_min[0, 0], real_alpha_max[0, 0])
    # First row (only x-1 is needed)
    if (y == 0):
        prevx = computeSector(x-1, y)
        real = intersect(prevx[0], prevx[1], base_alpha_min[y, x], base_alpha_max[y, x])
        return setRealAlpha(x, y, real[0], real[1])
    # First column (only y-1 is needed)
    if (x == 0):
        prevy = computeSector(x, y-1)
        real = intersect(prevy[0], prevy[1], base_alpha_min[y, x], base_alpha_max[y, x])
        return setRealAlpha(x, y, real[0], real[1])
    # The other cells (x-1 AND y-1 control the flow of light entering this cell)
    else:
        prevx = computeSector(x-1, y)
        prevy = computeSector(x, y-1)
        real = []
        btype = ""
        if (blocks[y, x]==1 and blocks[y, x-1]==1):
            btype="h"
            real = intersect(prevy[0], prevy[1], base_alpha_min_h[y, x], base_alpha_max_h[y, x])
            print(f"({x},{y}) blocks H : real({real[0], real[1]})")
        elif (blocks[y, x]==1 and blocks[y-1, x]==1):
            btype="v"
            real = intersect(prevx[0], prevx[1], base_alpha_min_v[y, x], base_alpha_max_v[y, x])
            print(f"({x},{y}) blocks V : real({real[0], real[1]})")
        else:    
            real = intersect(prevx[0], prevx[1], base_alpha_min[y, x], base_alpha_max[y, x])
            real2 = intersect(prevy[0], prevy[1], base_alpha_min[y, x], base_alpha_max[y, x])
            real = union(real[0], real[1], real2[0], real2[1])
        if (btype!="") :
            if (blocks[y-1, x] == blocks[y, x-1] == 1):
                print(f"({x},{y}) angle")
                btype="a"
        return setRealAlpha(x, y, real[0], real[1], btype)


def convertLightingRatioToColor():
    for y in range(HEIGHT):
        for x in range(WIDTH):
            if (real_ratio[y, x] == 0.):
                lighting_color[y, x] = 0
            else:
                lighting_color[y, x] = int(abs(real_ratio[y, x]) * COLOR_RANGE) * COLOR_DELTA + COLOR_MIN

def color_max_red(value):
    v = int(value)
    hexcolor = '#%02x%02x%02x' % (v, v, v)
    return 'background-color: {}'.format(hexcolor)

def color_block(value):
    return 'font-weight: bold; color: red'

def markBlocks(datastyle):
    for y in range(HEIGHT):
        for x in range(WIDTH):
            if (blocks[y, x]):
                datastyle.applymap(color_block, subset=(y, x))

def fakeDisplay(msg=""):
    df = pd.DataFrame(lighting_color).sort_index(ascending=False, axis=0)
    s = df.style.applymap(color_max_red)
    markBlocks(s)
    if msg!="":
        display(HTML("<b>"+msg+"</b>"))
    display(HTML(s.render()+"</br>"))

print("Functions: OK")  

Functions: OK


In [6]:
computeBaseSector()
computeBaseSector_Walls()
print("Base sectors: OK")

Base sectors: OK


### Situation de départ sans aucun obstacle

In [7]:
resetArrays()

computeSector(WIDTH-1, HEIGHT-1)
convertLightingRatioToColor()

print(f"Light source angles ({light_angles[0]}, {light_angles[1]})\n")
#display("Real Ratio", real_ratio)    
fakeDisplay("Real Ratio to color")

Light source angles (-180.0, 180.0)



,0,1,2,3,4,5,6,7,8,9
9,200,200,200,200,200,200,200,200,200,200
8,200,200,200,200,200,200,200,200,200,200
7,200,200,200,200,200,200,200,200,200,200
6,200,200,200,200,200,200,200,200,200,200
5,200,200,200,200,200,200,200,200,200,200
4,200,200,200,200,200,200,200,200,200,200
3,200,200,200,200,200,200,200,200,200,200
2,200,200,200,200,200,200,200,200,200,200
1,200,200,200,200,200,200,200,200,200,200
0,200,200,200,200,200,200,200,200,200,200


Calcul des distances Cellule à Source lumineuse

In [8]:
dist = np.zeros([HEIGHT, WIDTH])
lum = np.zeros([HEIGHT, WIDTH])

def computeBaseDistance():
    for y in range(HEIGHT):
        for x in range (WIDTH):
            a = np.array([x+0.5, y+0.5]) - L
            dist[y, x] = abs(np.linalg.norm([a]))
    # to avoid special case at origin
    dist[0, 0] = 1.

computeBaseDistance()
dumpdata("distances", dist)   

## distances ##
      0         1         2         3         4          5          6  \
9  9.0  9.055385  9.219544  9.486833  9.848858  10.295630  10.816654   
8  8.0  8.062258  8.246211  8.544004  8.944272   9.433981  10.000000   
7  7.0  7.071068  7.280110  7.615773  8.062258   8.602325   9.219544   
6  6.0  6.082763  6.324555  6.708204  7.211103   7.810250   8.485281   
5  5.0  5.099020  5.385165  5.830952  6.403124   7.071068   7.810250   
4  4.0  4.123106  4.472136  5.000000  5.656854   6.403124   7.211103   
3  3.0  3.162278  3.605551  4.242641  5.000000   5.830952   6.708204   
2  2.0  2.236068  2.828427  3.605551  4.472136   5.385165   6.324555   
1  1.0  1.414214  2.236068  3.162278  4.123106   5.099020   6.082763   
0  1.0  1.000000  2.000000  3.000000  4.000000   5.000000   6.000000   

           7          8          9  
9  11.401754  12.041595  12.727922  
8  10.630146  11.313708  12.041595  
7   9.899495  10.630146  11.401754  
6   9.219544  10.000000  10.816654  
5   8

## 2. Atténuation quadratique

### 2.1 quadratique pure

In [9]:
lum = np.zeros([HEIGHT, WIDTH])

# 1/d^2 falloff
def computeLighting_d2falloff():
    for y in range(HEIGHT):
        for x in range (WIDTH):
            lum[y, x] = real_ratio[y, x] / (dist[y, x]*dist[y, x])

def convertRatioToColor(source):
    for y in range(HEIGHT):
        for x in range(WIDTH):
            if (source[y, x] == 0.):
                lighting_color[y, x] = 0
            else:
                lighting_color[y, x] = int(abs(source[y, x]) * COLOR_RANGE) * COLOR_DELTA + COLOR_MIN

def convertRatioToRGB(source):
    for y in range(HEIGHT):
        for x in range(WIDTH):
            if (source[y, x] == 0.):
                lighting_color[y, x] = 0
            else:
                lighting_color[y, x] = int(abs(source[y, x]) * 255)

def fakeDisplay2():
    df = pd.DataFrame(lighting_color).sort_index(ascending=False, axis=0)
    s = df.style.applymap(color_max_red)
    markBlocks(s)
#    dumpdata("blocks", blocks)
    return(df)

resetArrays()
computeSector(WIDTH-1, HEIGHT-1)

computeLighting_d2falloff()
dumpdata("lum", lum)

convertRatioToColor(lum)
fakeDisplay("pure quadratic falloff")

convertRatioToRGB(lum)
fakeDisplay("pure quadratic falloff (RGB)")



## lum ##
           0         1         2         3         4         5         6  \
9  0.012346  0.012195  0.011765  0.011111  0.010309  0.009434  0.008547   
8  0.015625  0.015385  0.014706  0.013699  0.012500  0.011236  0.010000   
7  0.020408  0.020000  0.018868  0.017241  0.015385  0.013514  0.011765   
6  0.027778  0.027027  0.025000  0.022222  0.019231  0.016393  0.013889   
5  0.040000  0.038462  0.034483  0.029412  0.024390  0.020000  0.016393   
4  0.062500  0.058824  0.050000  0.040000  0.031250  0.024390  0.019231   
3  0.111111  0.100000  0.076923  0.055556  0.040000  0.029412  0.022222   
2  0.250000  0.200000  0.125000  0.076923  0.050000  0.034483  0.025000   
1  1.000000  0.500000  0.200000  0.100000  0.058824  0.038462  0.027027   
0  1.000000  1.000000  0.250000  0.111111  0.062500  0.040000  0.027778   

          7         8         9  
9  0.007692  0.006897  0.006173  
8  0.008850  0.007812  0.006897  
7  0.010204  0.008850  0.007692  
6  0.011765  0.010000  0.00

,0,1,2,3,4,5,6,7,8,9
9,40,40,40,40,40,40,40,40,40,40
8,40,40,40,40,40,40,40,40,40,40
7,40,40,40,40,40,40,40,40,40,40
6,40,40,40,40,40,40,40,40,40,40
5,40,40,40,40,40,40,40,40,40,40
4,40,40,40,40,40,40,40,40,40,40
3,56,40,40,40,40,40,40,40,40,40
2,72,56,56,40,40,40,40,40,40,40
1,200,104,56,40,40,40,40,40,40,40
0,200,200,72,56,40,40,40,40,40,40


,0,1,2,3,4,5,6,7,8,9
9,3,3,3,2,2,2,2,1,1,1
8,3,3,3,3,3,2,2,2,1,1
7,5,5,4,4,3,3,3,2,2,1
6,7,6,6,5,4,4,3,3,2,2
5,10,9,8,7,6,5,4,3,2,2
4,15,15,12,10,7,6,4,3,3,2
3,28,25,19,14,10,7,5,4,3,2
2,63,50,31,19,12,8,6,4,3,3
1,255,127,50,25,15,9,6,5,3,3
0,255,255,63,28,15,10,7,5,3,3


#### constatations :

- l'atténuation est _très_ rapide
- la zone à la limite du visible est très étendue et il faudra la limiter au passage à la version "discrète" avec n niveaux de gris
- il faudrait répartir les niveaux de gris de manière non linéaire pour s'adapter à la perception humaine et privilégier les ombres (attention à la correction gamma)

### 2.2 quadratique avec constante et limite

In [10]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

print("interact: OK")

interact: OK


In [11]:
# k/d^2 falloff + max clamp
def computeLighting_d2falloff_kclamp(k=5):
    for y in range(HEIGHT):
        for x in range (WIDTH):
            lum[y, x] = min(k*(real_ratio[y, x] / (dist[y, x]*dist[y, x])), 1.)

def proceed(k):
    computeLighting_d2falloff_kclamp(k)
    #display("lum", lum)
    convertRatioToColor(lum)
    fakeDisplay(f"quadratic falloff with constant k={k} and limit")
    
    convertRatioToRGB(lum)
    fakeDisplay(f"quadratic falloff with constant k={k} and limit (RGB)")    
    
resetArrays()
computeSector(WIDTH-1, HEIGHT-1)

if INTERACTIVE:
    interact(proceed, k=widgets.FloatSlider(min=0.5,max=50.,step=0.5,value=5.));    
else:
    k = 7.
    proceed(k)

,0,1,2,3,4,5,6,7,8,9
9,40,40,40,40,40,40,40,40,40,40
8,56,56,56,40,40,40,40,40,40,40
7,56,56,56,56,56,40,40,40,40,40
6,56,56,56,56,56,56,40,40,40,40
5,72,72,72,72,56,56,56,40,40,40
4,104,104,88,72,72,56,56,56,40,40
3,152,136,120,88,72,72,56,56,40,40
2,200,200,168,120,88,72,56,56,56,40
1,200,200,200,136,104,72,56,56,56,40
0,200,200,200,152,104,72,56,56,56,40


,0,1,2,3,4,5,6,7,8,9
9,22,21,21,19,18,16,15,13,12,11
8,27,27,26,24,22,20,17,15,13,12
7,36,35,33,30,27,24,21,18,15,13
6,49,48,44,39,34,29,24,21,17,15
5,71,68,61,52,43,35,29,24,20,16
4,111,105,89,71,55,43,34,27,22,18
3,198,178,137,99,71,52,39,30,24,19
2,255,255,223,137,89,61,44,33,26,21
1,255,255,255,178,105,68,48,35,27,21
0,255,255,255,198,111,71,49,36,27,22


#### Avec des obstacles

In [12]:
resetArrays()

blocks[3, 2] = 1
blocks[2, 3] = 1
blocks[3, 3] = 1

blocks[0, 4] = 1

computeSector(WIDTH-1, HEIGHT-1)

if INTERACTIVE:
    interact(proceed, k=widgets.FloatSlider(min=0.5,max=50.,step=0.5,value=5.));
else:
    k = 7.
    proceed(k)

(3,3) blocks H : real((0.0, 0.0))
(3,3) angle


,0,1,2,3,4,5,6,7,8,9
9,40,40,40,40,40,0,0,0,0,0
8,56,56,56,40,40,0,0,0,0,0
7,56,56,56,40,0,0,0,0,0,0
6,56,56,56,40,0,0,0,0,0,0
5,72,72,56,0,0,0,0,0,0,0
4,104,104,40,0,0,0,0,0,40,40
3,152,136,120,88,0,0,40,40,40,40
2,200,200,168,120,40,56,56,56,56,40
1,200,200,200,136,88,72,56,40,40,40
0,200,200,200,152,104,0,0,0,0,0


,0,1,2,3,4,5,6,7,8,9
9,22,21,21,19,7,0,0,0,0,0
8,27,27,26,19,2,0,0,0,0,0
7,36,35,33,14,0,0,0,0,0,0
6,49,48,39,7,0,0,0,0,0,0
5,71,68,35,0,0,0,0,0,0,0
4,111,105,24,0,0,0,0,0,2,7
3,198,178,137,99,0,0,7,14,19,19
2,255,255,223,137,24,35,39,33,26,21
1,255,255,255,178,93,53,31,18,10,5
0,255,255,255,198,111,0,0,0,0,0


### 2.3 quadratique et diamètre min/max

In [13]:
# k/d^2 falloff + max clamp
def computeLighting_quadfalloff_minmaxradius(rmin=2, rmax=3):
    delta = (rmax - rmin)
    for y in range(HEIGHT):
        for x in range (WIDTH):
            if dist[y, x] <= rmin:
                lum[y, x] = real_ratio[y, x]
            else:
                vx = dist[y, x] - rmin
                if vx < delta:
                    lum[y, x] = max(real_ratio[y, x] * ((vx/delta -1)*(vx/delta -1)), 0.)
                else:
                    lum[y, x] = 0.
                
                
def proceed(rmin, rmax):
    computeLighting_quadfalloff_minmaxradius(rmin, rmax)
    convertRatioToColor(lum)
    fakeDisplay(f"max with distance in [0, rmin={rmin}] and quadratic falloff in ]rmin={rmin}, rmax={rmax}]")
    
    convertRatioToRGB(lum)
    fakeDisplay(f"max with distance in [0, rmin={rmin}] and quadratic falloff in ]rmin={rmin}, rmax={rmax}] (RGB)")    

wrmin, wrmax = widgets.FloatSlider(min=2.,max=10.,step=0.5,value=2.),\
                widgets.FloatSlider(min=3.,max=10.,step=0.5,value=7.5)

def update_rmax_range(*args):
    wrmax.min = wrmin.value + 0.5

resetArrays()
computeSector(WIDTH-1, HEIGHT-1)

if INTERACTIVE:
    wrmin.observe(update_rmax_range, 'value')
    interact(proceed, rmin=wrmin, rmax=wrmax)
else:
    rmin=2
    rmax=7.5
    proceed(rmin, rmax)

,0,1,2,3,4,5,6,7,8,9
9,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
7,40,40,40,0,0,0,0,0,0,0
6,40,40,40,40,40,0,0,0,0,0
5,72,56,56,40,40,40,0,0,0,0
4,104,88,88,72,56,40,40,0,0,0
3,136,136,120,88,72,40,40,0,0,0
2,200,184,152,120,88,56,40,40,0,0
1,200,200,184,136,88,56,40,40,0,0
0,200,200,200,136,104,72,40,40,0,0


,0,1,2,3,4,5,6,7,8,9
9,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
7,2,1,0,0,0,0,0,0,0,0
6,18,16,11,5,0,0,0,0,0,0
5,52,48,37,23,10,1,0,0,0,0
4,103,96,77,52,28,10,0,0,0,0
3,170,158,127,89,52,23,5,0,0,0
2,255,233,183,127,77,37,11,0,0,0
1,255,255,233,158,96,48,16,1,0,0
0,255,255,255,170,103,52,18,2,0,0


#### Avec des obstacles

In [14]:
resetArrays()

blocks[3, 2] = 1
blocks[2, 3] = 1
blocks[3, 3] = 1

blocks[0, 4] = 1

computeSector(WIDTH-1, HEIGHT-1)

if INTERACTIVE:
    interact(proceed, rmin=wrmin, rmax=wrmax)
else:
    rmin=2.
    rmax=7.5
    proceed(rmin, rmax)

(3,3) blocks H : real((0.0, 0.0))
(3,3) angle


,0,1,2,3,4,5,6,7,8,9
9,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
7,40,40,40,0,0,0,0,0,0,0
6,40,40,40,40,0,0,0,0,0,0
5,72,56,40,0,0,0,0,0,0,0
4,104,88,40,0,0,0,0,0,0,0
3,136,136,120,88,0,0,40,0,0,0
2,200,184,152,120,40,40,40,40,0,0
1,200,200,184,136,88,56,40,40,0,0
0,200,200,200,136,104,0,0,0,0,0


,0,1,2,3,4,5,6,7,8,9
9,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
7,2,1,0,0,0,0,0,0,0,0
6,18,16,10,0,0,0,0,0,0,0
5,52,48,21,0,0,0,0,0,0,0
4,103,96,21,0,0,0,0,0,0,0
3,170,158,127,89,0,0,0,0,0,0
2,255,233,183,127,21,21,10,0,0,0
1,255,255,233,158,85,37,11,0,0,0
0,255,255,255,170,103,0,0,0,0,0


## 3. Atténuation linéaire et diamètre min/max

In [15]:
resetArrays()
computeSector(WIDTH-1, HEIGHT-1)

# k/d^2 falloff + max clamp
def computeLighting_linfalloff_minmaxradius(rmin=2, rmax=3):
    delta = abs(rmax - rmin)
    for y in range(HEIGHT):
        for x in range (WIDTH):
            if dist[y, x] <= rmin:
                lum[y, x] = real_ratio[y, x]
            else:
                lum[y, x] = max(real_ratio[y, x] - ( (real_ratio[y, x]/delta) * (dist[y, x] - rmin)), 0.)

def proceed(rmin, rmax):
    computeLighting_linfalloff_minmaxradius(rmin, rmax)
    convertRatioToColor(lum)
    fakeDisplay(f"max with distance in [0, rmin={rmin}] and linear falloff in ]rmin={rmin}, rmax={rmax}]")
    convertRatioToRGB(lum)
    fakeDisplay(f"max with distance in [0, rmin={rmin}] and linear falloff in ]rmin={rmin}, rmax={rmax}]")

wrmin, wrmax = widgets.FloatSlider(min=2.,max=10.,step=0.5,value=2.),\
                widgets.FloatSlider(min=3.,max=10.,step=0.5,value=7.5)

def update_rmax_range(*args):
    wrmax.min = wrmin.value + 0.5

resetArrays()
computeSector(WIDTH-1, HEIGHT-1)

if INTERACTIVE:
    wrmin.observe(update_rmax_range, 'value')
    interact(proceed, rmin=wrmin, rmax=wrmax)
else:
    rmin=2.
    rmax=7.5
    proceed(rmin, rmax)

,0,1,2,3,4,5,6,7,8,9
9,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
7,40,40,40,0,0,0,0,0,0,0
6,72,72,72,56,40,0,0,0,0,0
5,104,104,88,88,56,40,0,0,0,0
4,136,136,120,104,88,56,40,0,0,0
3,168,152,152,120,104,88,56,0,0,0
2,200,184,168,152,120,88,72,40,0,0
1,200,200,184,152,136,104,72,40,0,0
0,200,200,200,168,136,104,72,40,0,0


,0,1,2,3,4,5,6,7,8,9
9,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
7,23,19,10,0,0,0,0,0,0,0
6,69,65,54,36,13,0,0,0,0,0
5,115,111,98,77,50,19,0,0,0,0
4,162,156,140,115,85,50,13,0,0,0
3,208,201,180,151,115,77,36,0,0,0
2,255,244,216,180,140,98,54,10,0,0
1,255,255,244,201,156,111,65,19,0,0
0,255,255,255,208,162,115,69,23,0,0


In [16]:
resetArrays()

blocks[3, 2] = 1
blocks[2, 3] = 1
blocks[3, 3] = 1

blocks[0, 4] = 1

computeSector(WIDTH-1, HEIGHT-1)

if INTERACTIVE:
    interact(proceed, rmin=wrmin, rmax=wrmax)
else:
    rmin = 2.
    rmax = 7.5
    proceed(rmin, rmax)   

(3,3) blocks H : real((0.0, 0.0))
(3,3) angle


,0,1,2,3,4,5,6,7,8,9
9,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
7,40,40,40,0,0,0,0,0,0,0
6,72,72,56,40,0,0,0,0,0,0
5,104,104,72,0,0,0,0,0,0,0
4,136,136,56,0,0,0,0,0,0,0
3,168,152,152,120,0,0,40,0,0,0
2,200,184,168,152,56,72,56,40,0,0
1,200,200,184,152,120,88,56,40,0,0
0,200,200,200,168,136,0,0,0,0,0


,0,1,2,3,4,5,6,7,8,9
9,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
7,23,19,10,0,0,0,0,0,0,0
6,69,65,48,6,0,0,0,0,0,0
5,115,111,56,0,0,0,0,0,0,0
4,162,156,39,0,0,0,0,0,0,0
3,208,201,180,151,0,0,6,0,0,0
2,255,244,216,180,39,56,48,10,0,0
1,255,255,244,201,139,86,43,10,0,0
0,255,255,255,208,162,0,0,0,0,0
